In [1]:
#!pip install instaloader

In [2]:
import instaloader
from datetime import datetime
from itertools import dropwhile, takewhile
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from string import punctuation

from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

from nltk.stem import PorterStemmer
porter = PorterStemmer()

from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\linzh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
L = instaloader.Instaloader()

In [4]:
# not using now due to failure to login

with open('../login_info.txt', encoding='utf-8') as file:
    content = file.read()
    username = content.split()[0]
    pswd = content.split()[1]

# L.login(username, pswd)
# L.save_session_to_file('session_file')

Default value (read from 'ins_account_usernames.txt') for account_usernames is a list of:
["NBCNews", 
 "FoxNews", 
 "CNN", 
 "NYTimes",
 "WashTimes", 
 "WSJ", 
 "BBCNews", 
 "Reuters",
 "Time", 
 "Aljazeeraenglish", 
 "Tabletmag", 
 "voxdotcom",
 "vicenews", 
 "Axios", 
 "HuffPost"]

In [5]:
with open('ins_account_usernames.txt', encoding='utf-8') as file:
    content = file.read()
    ins_usernames_list = content.split()

For details of properties of Post class:\
https://instaloader.github.io/module/structures.html#posts

For details of properties of Profile class:\
https://instaloader.github.io/module/structures.html?highlight=postcomment#profiles


In [6]:
# for username in account_usernames:
#     df_name = "df_" + username
#     exec_string = "{}.to_csv('latest_ins_csv/{}.csv', index=False)"
#     exec(exec_string.format(df_name, df_name))

**NOTE for this following cell**:\
If successfully scraped for all usernames (no query error popped up), save the exported csv files into a new folder.\
If failed, just skip this cell and run the next cell.

In [7]:
# lists to generate summary info of basic account info
userid_list = []
followers_list = []
full_name_list = []
is_verified_list = []
post_count_list = []    # counts of posts since 231001

for username in ins_usernames_list:
    posts = instaloader.Profile.from_username(L.context, username).get_posts()

    SINCE = datetime(2023, 10, 1)
    UNTIL = datetime.today()
    
    url_code_list = []
    time_utc_list = []
    content_types_list = []
    captions_list = []
    like_counts_list = []
    comment_counts_list = []

    for post in takewhile(lambda p: p.date > SINCE, dropwhile(lambda p: p.date > UNTIL, posts)):
    
        # details of post: date&time, count of likes, count of comments, url
        url_code_list.append(post.shortcode)
        time_utc_list.append(post.date_utc)
        content_types_list.append(post.typename)
        captions_list.append(post.caption)
        like_counts_list.append(post.likes)
        comment_counts_list.append(post.comments)
        # L.download_post(post, username)
    
    # create df with info of each posts from this account
    df_name = "df_" + username
    exec_string = """{} = pd.DataFrame(data={{
    'owner_username': [username] * len(url_code_list),
    'url_code': url_code_list, 
    'time_utc': pd.to_datetime(time_utc_list), 
    'type': content_types_list,
    'caption': captions_list,
    'likes': like_counts_list, 
    'comments': comment_counts_list}})
    """
    exec(exec_string.format(df_name))
    
    # export df to csv
    exec_string = "{}.to_csv('latest_ins_csv/{}.csv', index=False)"
    exec(exec_string.format(df_name, df_name))
    
    print("Done with posts of", username)
    print("{} has {} posts since October 1st.".format(username, len(url_code_list)))
    post_count_list.append(len(url_code_list))
    
    
    # append this account info
    userid_list.append(post.owner_profile.userid)
    followers_list.append(post.owner_profile.followers)
    full_name_list.append(post.owner_profile.full_name)
    is_verified_list.append(post.owner_profile.is_verified)
    
d = {'user_id': userid_list, 'followers': followers_list, 'full_name':full_name_list, 'is_verified':is_verified_list, 'posts_since_231001': post_count_list}
df_ins_accounts = pd.DataFrame(data=d)
df_ins_accounts.to_csv('latest_ins_csv/df_ins_accounts.csv', index=False)

Done with posts of NBCNews
NBCNews has 117 posts since October 1st.
Done with posts of FoxNews
FoxNews has 614 posts since October 1st.
Done with posts of CNN
CNN has 167 posts since October 1st.
Done with posts of NYTimes
NYTimes has 189 posts since October 1st.
Done with posts of WashTimes
WashTimes has 170 posts since October 1st.
Done with posts of WSJ
WSJ has 75 posts since October 1st.
Done with posts of BBCNews
BBCNews has 178 posts since October 1st.
Done with posts of Reuters
Reuters has 54 posts since October 1st.
Done with posts of Time
Time has 90 posts since October 1st.
Done with posts of Aljazeeraenglish
Aljazeeraenglish has 581 posts since October 1st.
Done with posts of Tabletmag
Tabletmag has 50 posts since October 1st.
Done with posts of voxdotcom
voxdotcom has 41 posts since October 1st.
Done with posts of vicenews
vicenews has 70 posts since October 1st.
Done with posts of Axios
Axios has 62 posts since October 1st.
Done with posts of HuffPost
HuffPost has 31 posts

In [7]:
stacked_dfs = pd.DataFrame()

try:
    if len(df_ins_accounts) == len(ins_usernames_list):
        print("Successfully scraped for all usernames.")
        print("Remember to save the exported csv files in the directory of './latest_ins_csv' into a new folder")
        
        for username in ins_usernames_list:
            df_name = "df_" + username
            exec("stacked_dfs=pd.concat([stacked_dfs, {}], ignore_index=True)".format(df_name))
            
    else:
        print("Not all usernames were successfully scraped.")
        print("Use backup files instead.")
        
        df_ins_accounts = pd.read_csv('backup_df_231025/df_ins_accounts.csv')
        for username in ins_usernames_list:
            df_name = "df_" + username
            path = 'backup_df_231025/' + df_name + ".csv"
            exec("stacked_dfs=pd.concat([stacked_dfs, pd.read_csv('{}')], ignore_index=True)".format(path))

except:
    print("Failed to scrape.")
    print("Use backup files instead.")
    
    df_ins_accounts = pd.read_csv('backup_df_231025/df_ins_accounts.csv')
    for username in ins_usernames_list:
        df_name = "df_" + username
        path = 'backup_df_231025/' + df_name + ".csv"
        exec("stacked_dfs=pd.concat([stacked_dfs, pd.read_csv('{}')], ignore_index=True)".format(path))

Failed to scrape.
Use backup files instead.


In [8]:
# from HW2
def find_freq_tokens(tokens, display_limit):
    
    freq_dict = FreqDist(tokens)
    bigram_freq = dict()
    
    sorted_items = sorted(freq_dict.items(), key=lambda item: item[1], reverse=True)
    top_keys = [item[0] for item in sorted_items[:display_limit]]
    
    return top_keys

In [9]:
def text_to_tokens(text):
    
    lower_str = text.lower()
    no_punc_str = ''.join([character for character in lower_str if character not in punctuation])
    no_digit_str = ''.join([character for character in no_punc_str if not character.isdigit()])
    no_special_charac = ''.join([character for character in no_digit_str if character not in ['\u2060', '”', '“']])
    no_dash = no_special_charac.replace('—', ' ').replace('-', ' ').replace('’', ' ')
    orig_tokens = word_tokenize(no_dash)
    
    tokens = [porter.stem(word) for word in orig_tokens if not word in stop_words]
    
    return tokens

In [10]:
stacked_dfs = stacked_dfs.dropna(subset=['caption'])

In [11]:
all_ins_tokens = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])
    all_ins_tokens += tokens

top_tokens = find_freq_tokens(all_ins_tokens, 30)
top_tokens

['link',
 'bio',
 'israel',
 'gaza',
 'isra',
 'hama',
 'said',
 'peopl',
 'palestinian',
 'attack',
 'read',
 'kill',
 'tap',
 'us',
 'say',
 'time',
 'palestin',
 'year',
 'war',
 'one',
 'new',
 'israelgazawar',
 'bbcnew',
 'live',
 'citi',
 'hospit',
 'world',
 'hous',
 'al',
 'presid']

## Pick out the posts that talks about gaza conflict
- method 1: find the posts that contain at least one of the tokens in gaza_tokens_1
- method 2: find the posts that contain at least one of the tokens in gaza_tokens_2, which has two more tokens: "war" and "attack"
- method 3: use a language model to pick

**method 1**

In [12]:
gaza_tokens_1 = ['israel',
 'gaza',
 'isra',
 'hama',
 'palestinian',
 'palestin',
 'israelgazawar',
 'israelhama',
 'israelgaza']


is_about_gaza_1 = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])

    found = False
    for g_token in gaza_tokens_1:
        if g_token in tokens:
            found = True
            break

    is_about_gaza_1.append(found)

stacked_dfs['is_about_gaza_1'] = is_about_gaza_1

stacked_dfs[stacked_dfs['is_about_gaza_1']==True]

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1
0,NBCNews,Cy0_M9PLRUw,2023-10-25 16:08:27,GraphVideo,Inside Al-Shifa hospital in Gaza as officials ...,2668,686,True
1,NBCNews,Cy07NokrYeK,2023-10-25 15:33:09,GraphImage,"With so many bodies, Palestinians in Gaza are ...",2819,521,True
3,NBCNews,CyzC0xvSAgD,2023-10-24 22:01:21,GraphVideo,"A doctor at Al-Shifa Hospital, the largest med...",2352,436,True
4,NBCNews,Cyy7-5VvN5J,2023-10-24 21:01:23,GraphSidecar,"Clean drinking water is running out in Gaza, f...",6319,917,True
5,NBCNews,CyydQhNrBDa,2023-10-24 16:33:01,GraphVideo,A man in Khan Younis says 50 members of his fa...,2964,894,True
...,...,...,...,...,...,...,...,...
2467,HuffPost,CydlouMNpDf,2023-10-16 14:02:10,GraphImage,A 6-year-old boy and his mother were stabbed o...,3327,459,True
2469,HuffPost,CyWOe-YJr-F,2023-10-13 17:25:13,GraphImage,"As Israel escalates its attacks on Gaza, the S...",9068,1102,True
2470,HuffPost,CyWDC1_MbYm,2023-10-13 15:45:13,GraphImage,A BBC Arabic journalist reported on the distre...,2499,254,True
2471,HuffPost,CyV60HvNHf6,2023-10-13 14:33:12,GraphSidecar,When Neta Heiman last spoke to her 84-year-old...,11172,310,True


In [13]:
all_ins_GAZA_tokens = []

for index, row in stacked_dfs[stacked_dfs['is_about_gaza_1']==True].iterrows():

    tokens = text_to_tokens(row['caption'])
    all_ins_GAZA_tokens += tokens

len(all_ins_GAZA_tokens)

50729

In [14]:
all_ins_GAZA_tokens = []

for index, row in stacked_dfs[stacked_dfs['is_about_gaza_1']==True].iterrows():

    tokens = word_tokenize(row['caption'])
    all_ins_GAZA_tokens += tokens

len(all_ins_GAZA_tokens)

100253

**method 2**

In [15]:
gaza_tokens_2 = ['israel',
 'gaza',
 'isra',
 'hama',
 'palestinian',
 'attack',
 'palestin',
 'war',
 'israelgazawar',
 'israelhama',
 'israelgaza']


is_about_gaza_2 = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])

    found = False
    for g_token in gaza_tokens_2:
        if g_token in tokens:
            found = True
            break

    is_about_gaza_2.append(found)

stacked_dfs['is_about_gaza_2'] = is_about_gaza_2

stacked_dfs[stacked_dfs['is_about_gaza_2']==True]

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1,is_about_gaza_2
0,NBCNews,Cy0_M9PLRUw,2023-10-25 16:08:27,GraphVideo,Inside Al-Shifa hospital in Gaza as officials ...,2668,686,True,True
1,NBCNews,Cy07NokrYeK,2023-10-25 15:33:09,GraphImage,"With so many bodies, Palestinians in Gaza are ...",2819,521,True,True
3,NBCNews,CyzC0xvSAgD,2023-10-24 22:01:21,GraphVideo,"A doctor at Al-Shifa Hospital, the largest med...",2352,436,True,True
4,NBCNews,Cyy7-5VvN5J,2023-10-24 21:01:23,GraphSidecar,"Clean drinking water is running out in Gaza, f...",6319,917,True,True
5,NBCNews,CyydQhNrBDa,2023-10-24 16:33:01,GraphVideo,A man in Khan Younis says 50 members of his fa...,2964,894,True,True
...,...,...,...,...,...,...,...,...,...
2467,HuffPost,CydlouMNpDf,2023-10-16 14:02:10,GraphImage,A 6-year-old boy and his mother were stabbed o...,3327,459,True,True
2469,HuffPost,CyWOe-YJr-F,2023-10-13 17:25:13,GraphImage,"As Israel escalates its attacks on Gaza, the S...",9068,1102,True,True
2470,HuffPost,CyWDC1_MbYm,2023-10-13 15:45:13,GraphImage,A BBC Arabic journalist reported on the distre...,2499,254,True,True
2471,HuffPost,CyV60HvNHf6,2023-10-13 14:33:12,GraphSidecar,When Neta Heiman last spoke to her 84-year-old...,11172,310,True,True


In [16]:
df_diff = stacked_dfs[stacked_dfs['is_about_gaza_1'] != stacked_dfs['is_about_gaza_2']]
df_diff

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1,is_about_gaza_2
35,NBCNews,CybrNO-LmsF,2023-10-15 20:12:15,GraphSidecar,Raised on her family’s farm in conservative ru...,3808,325,False,True
389,FoxNews,CyXrxHyN4Vh,2023-10-14 07:00:11,GraphImage,WARNING SHOT: Fox News host Sean Hannity offer...,15624,1489,False,True
407,FoxNews,CyV-JX4tMwa,2023-10-13 15:02:37,GraphVideo,TUNE IN: GOP presidential hopeful @nikkihaley ...,2661,187,False,True
454,FoxNews,CyRTNJGts_a,2023-10-11 19:30:07,GraphImage,NO HOLDS BARRED? The Jets quarterback appears ...,28980,663,False,True
531,FoxNews,CyJ3QU8upgE,2023-10-08 22:11:12,GraphImage,'SAVAGE ATTACK': Senator Tom Cotton delivers a...,20212,931,False,True
727,FoxNews,Cx1rHAQoGeQ,2023-10-01 02:00:16,GraphImage,'HENIOUS ATTACK': Lawyer of teenager assaulted...,29120,4175,False,True
736,CNN,CyzPktSJZqZ,2023-10-24 23:53:11,GraphVideo,Security footage shows three armed suspects in...,117088,5295,False,True
920,NYTimes,CytkqmLuuH9,2023-10-22 19:01:23,GraphSidecar,"Across Africa and in parts of Asia, disruption...",16471,1187,False,True
973,NYTimes,CyeBtSOJJLq,2023-10-16 18:08:10,GraphImage,Judge Tanya Chutkan on Monday barred Donald Tr...,13985,801,False,True
986,NYTimes,Cya61DJumtc,2023-10-15 13:10:10,GraphVideo,Migrants Chased and Snared in Razor Wire: A Ra...,21048,1327,False,True


In [17]:
len(df_diff)

50

In [18]:
for index, row in df_diff.iterrows():

    print(row['caption'])
    print("------------")

Raised on her family’s farm in conservative rural Nebraska in the 1960s and ‘70s, Ashley Swartz took her cues about life from her surroundings: At home, the men worked the farm; at play, boys roughhoused and participated in sports; at church, there was good or there was evil.

Swartz identified more with girls – wanting to play the female roles when playing make-believe and wishing to be considered pretty rather than handsome.

Swartz, like many older transgender Americans, had hoped their younger counterparts would not be faced with the same challenges and despair. But now, watching growing political attacks and legislative efforts targeting trans people, they are fearful about what the future holds for the next generation, particularly those living in politically conservative states.

Read more at the link in bio.

📷️ @madelinecass for @NBCNews
------------
WARNING SHOT: Fox News host Sean Hannity offers a blunt prediction for how the war will end.
------------
TUNE IN: GOP president

**Method 3**

In [19]:
is_about_gaza_3 = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])

    found = False
    for g_token in gaza_tokens_2:
        if g_token in tokens:
            found = True
            break

    is_about_gaza_3.append(found)

stacked_dfs['is_about_gaza_3'] = is_about_gaza_3

GAZA_WAR_stacked_dfs = stacked_dfs[stacked_dfs['is_about_gaza_3']==True]
GAZA_WAR_stacked_dfs

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1,is_about_gaza_2,is_about_gaza_3
0,NBCNews,Cy0_M9PLRUw,2023-10-25 16:08:27,GraphVideo,Inside Al-Shifa hospital in Gaza as officials ...,2668,686,True,True,True
1,NBCNews,Cy07NokrYeK,2023-10-25 15:33:09,GraphImage,"With so many bodies, Palestinians in Gaza are ...",2819,521,True,True,True
3,NBCNews,CyzC0xvSAgD,2023-10-24 22:01:21,GraphVideo,"A doctor at Al-Shifa Hospital, the largest med...",2352,436,True,True,True
4,NBCNews,Cyy7-5VvN5J,2023-10-24 21:01:23,GraphSidecar,"Clean drinking water is running out in Gaza, f...",6319,917,True,True,True
5,NBCNews,CyydQhNrBDa,2023-10-24 16:33:01,GraphVideo,A man in Khan Younis says 50 members of his fa...,2964,894,True,True,True
...,...,...,...,...,...,...,...,...,...,...
2467,HuffPost,CydlouMNpDf,2023-10-16 14:02:10,GraphImage,A 6-year-old boy and his mother were stabbed o...,3327,459,True,True,True
2469,HuffPost,CyWOe-YJr-F,2023-10-13 17:25:13,GraphImage,"As Israel escalates its attacks on Gaza, the S...",9068,1102,True,True,True
2470,HuffPost,CyWDC1_MbYm,2023-10-13 15:45:13,GraphImage,A BBC Arabic journalist reported on the distre...,2499,254,True,True,True
2471,HuffPost,CyV60HvNHf6,2023-10-13 14:33:12,GraphSidecar,When Neta Heiman last spoke to her 84-year-old...,11172,310,True,True,True


Skipping hashtag due to lack of useful posts from the limited and ramdomly fetched posts.

In [ ]:
# posts = instaloader.Instaloader().get_hashtag_posts("gaza")
# posts = instaloader.Hashtag.from_name(L.context, "gaza").get_top_posts()

Only top 9 posts of a hashtag can be fetched with the following method

In [ ]:
# posts = instaloader.Hashtag.from_name(L.context, "gaza").get_top_posts()